In [1]:
import numpy as np
C_OPTIONS = np.logspace(0,1,2)

In [15]:
C_OPTIONS

array([ 1., 10.])

In [13]:
np.logspace(0,2)

array([  1.        ,   1.09854114,   1.20679264,   1.32571137,
         1.45634848,   1.59985872,   1.75751062,   1.93069773,
         2.12095089,   2.32995181,   2.55954792,   2.8117687 ,
         3.0888436 ,   3.39322177,   3.72759372,   4.09491506,
         4.49843267,   4.94171336,   5.42867544,   5.96362332,
         6.55128557,   7.19685673,   7.90604321,   8.68511374,
         9.54095476,  10.48113134,  11.51395399,  12.64855217,
        13.89495494,  15.26417967,  16.76832937,  18.42069969,
        20.23589648,  22.22996483,  24.42053095,  26.82695795,
        29.47051703,  32.37457543,  35.56480306,  39.06939937,
        42.9193426 ,  47.14866363,  51.79474679,  56.89866029,
        62.50551925,  68.6648845 ,  75.43120063,  82.86427729,
        91.0298178 , 100.        ])

In [16]:
C_OPTIONS = np.logspace(0,2,3)

In [17]:
C_OPTIONS

array([  1.,  10., 100.])

In [60]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
import sklearn.feature_extraction.text as text
from sklearn import pipeline
from sklearn.metrics import precision_score, recall_score, accuracy_score

X, y = make_classification(
        n_samples=100, n_features=10, n_informative=2, n_clusters_per_class=1, 
        shuffle=False, random_state=42,shift=5)
chi2_stats, p_values = chi2(X, y)
chi2_stats

array([2.25415508e+01, 7.35179236e-02, 7.25973700e-02, 1.01320229e-02,
       1.08125275e+00, 5.54434465e-02, 1.13279935e-02, 1.09136519e-01,
       1.32228682e-01, 1.66505941e-02])

In [67]:
f_stats,f_p_values = f_classif(X, y)
f_stats

array([2.21202152e+02, 7.02422486e-01, 1.69733937e+00, 9.30724505e-01,
       5.41252016e+00, 3.24536409e-01, 4.70936314e-02, 5.72496149e-01,
       7.54385494e-01, 8.90000881e-02])

In [69]:
f_stats.argsort()[-3:]

array([2, 4, 0])

In [66]:
chi2_stats, chi2_p_values = chi2(X, y)
chi2_stats

array([2.25415508e+01, 7.35179236e-02, 7.25973700e-02, 1.01320229e-02,
       1.08125275e+00, 5.54434465e-02, 1.13279935e-02, 1.09136519e-01,
       1.32228682e-01, 1.66505941e-02])

In [85]:
from datetime import datetime
datetime.now().strftime("%A %d. %B %Y %H:%M:%S")

'Saturday 20. April 2024 08:13:29'

In [63]:
#X_reduced = X[:, chi2_stats.argsort()[-2:]]
chi2_stats.argsort()[-2:]

array([4, 0])

In [86]:
secs=46471
hrs,mins = secs//3600,(secs%3600)//60
hrs,mins

(12, 54)

In [87]:
12*3600 + 60 * 54

46440

In [70]:
import os.path
from sklearn.svm import LinearSVC



site = 'https://raw.githubusercontent.com/gawron/python-for-social-science/master/'\
'text_classification/'
#site = 'https://gawron.sdsu.edu/python_for_ss/course_core/book_draft/_static/'
df = pd.read_csv(os.path.join(site,"troll.csv"))
X,y = df["Comment"].values,df["Insult"].values

k=10_000
kb= SelectKBest(mutual_info_classif,k=k)
# You can try these others with this pipeline as well.
#kb= SelectKBest(f_classif,k=k)
#kb= SelectKBest(chi2,k=k)

T_train,T_test, y_train,y_test = train_test_split(X,y)
         
svm_clf = LinearSVC()

cv = text.CountVectorizer()
tf = text.TfidfTransformer(sublinear_tf=True)

pipeline_reg= pipeline.Pipeline([('vect', cv), ('feat_selector', kb),
                                ('tfidf',tf), ('svm', svm_clf)])

# Train
pipeline_reg.fit(T_train, y_train)
# Test
predicted = pipeline_reg.predict(T_test)

a,p, r = accuracy_score(predicted, y_test),\
                precision_score( y_test,predicted,),\
                        recall_score(y_test, predicted, )
print(f"{k=:>6_d} {a=:.3f} {p=:.3f} {r=:.3f}")

k=10_000 a=0.834 p=0.766 r=0.568
